# Dealing with imbalanced datasets, combining oversampling with VAE and undersampling to improve model recognition over all classes.  

Import packages, classifiers and etc.

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

from sklearn.metrics import confusion_matrix, make_scorer
from imblearn.metrics import classification_report_imbalanced, geometric_mean_score
from collections import Counter

from imblearn.over_sampling import SMOTE, RandomOverSampler

Import VAEOversampler.

In [2]:
from VAEOversampler import VAEOversampler

2023-07-11 00:54:45.749130: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-11 00:54:45.749156: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Loading data  
You can load some dataset from Imbalanced Learn list (https://imbalanced-learn.org/stable/datasets/index.html) or use your own data.  


In [3]:
from imblearn.datasets import fetch_datasets

dset_name = 'yeast_me2'
dset = fetch_datasets()[dset_name]

X, y = StandardScaler().fit_transform(dset.data), dset.target

In [4]:
X

array([[ 0.58198136,  0.88848148, -0.3466451 , ..., -0.0991314 ,
        -0.34417514, -0.5279193 ],
       [-0.51089067,  1.37281104, -0.23122636, ..., -0.0991314 ,
         0.52121948, -0.5279193 ],
       [ 1.01913017,  0.96920307, -0.11580762, ..., -0.0991314 ,
         0.52121948, -0.5279193 ],
       ...,
       [ 1.23770457,  0.5655951 , -1.61625127, ..., -0.0991314 ,
         1.04045625, -0.5279193 ],
       [-0.51089067, -0.80667199,  1.15379854, ..., -0.0991314 ,
         0.52121948,  1.06900494],
       [ 1.0919883 ,  0.32343032,  0.46128609, ..., -0.0991314 ,
         0.52121948, -0.5279193 ]])

In [5]:
y[y == -1] = 0

We split data into train and test partitions.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

This is a simple function to undersample freely.  

In [7]:
# RUS

def RUS(X_res, y_res, frac=1, minority_class_id=1, random_state=42):
    X_res = pd.DataFrame(X_res)
    X_res['Class'] = y_res

    X_neg = X_res[y_res != minority_class_id].sample(frac=frac, random_state=random_state)
    X_pos = X_res[y_res == minority_class_id].sample(frac=1, random_state=random_state)

    X_rus = pd.concat([X_neg, X_pos], ignore_index=True)

    X_eq = X_rus.drop('Class', axis=1)
    y_eq = X_rus['Class']

    return X_eq, y_eq

In [8]:
def train_val(X, y, Xt, yt, random_state=42):
    classifiers = {
        "CatBoostClassifier": CatBoostClassifier(verbose=False, random_seed=random_state),
        "LGBMClassifier": LGBMClassifier(random_state=random_state),
        "XGBClassifier": XGBClassifier(random_state=random_state),
        "BaggingClassifier": BaggingClassifier(random_state=random_state),
        "RandomForestClassifier": RandomForestClassifier(random_state=random_state),
    }
    scores = []
    predictions = []
    for key, classifier in classifiers.items():
        print('_' * 50)
        name = key
        classifier.fit(X, y)
        print("Classifier: ", name)
        y_pred = classifier.predict(Xt)
        cm = confusion_matrix(yt, y_pred)
        print(cm)
        print('')
        predictions.append(y_pred)
        tn = cm[0,0]
        fp = cm[0,1]
        fn = cm[1,0]
        tp = cm[1,1]
        tnr = tn / (tn + fp)
        tpr = tp / (tp + fn)
        scores.append(tnr * tpr)
        print('TNR:', round(tnr, 5))
        print('TPR:', round(tpr, 5))
        print('TNRxTPR:', round(tnr * tpr, 5))
        print('G-mean:', round(np.sqrt(tnr * tpr), 5))

    print('_' * 50)
    print('Ensemble predictions (majority voting):')
    predictions = np.sum(predictions, axis=0)
    predictions[predictions < 3] = 0
    predictions[predictions >= 3] = 1

    cm = confusion_matrix(yt, predictions)
    print(cm)
    tn = cm[0,0]
    fp = cm[0,1]
    fn = cm[1,0]
    tp = cm[1,1]
    tnr = tn / (tn + fp)
    tpr = tp / (tp + fn)
    print('')
    print('TNR:', round(tnr, 5))
    print('TPR:', round(tpr, 5))
    print('TNRxTPR:', round(tnr * tpr, 5))
    print('G-mean:', round(np.sqrt(tnr * tpr), 5))


## Without resampling (base line) 
Which is starting score?


In [9]:
print('Original dataset shape %s' % Counter(y))
print('Ratio->', round(Counter(y)[0]/Counter(y)[1], 1), ': 1')

Original dataset shape Counter({0: 1433, 1: 51})
Ratio-> 28.1 : 1


In [10]:
train_val(X_train, y_train, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[284   3]
 [  7   3]]

TNR: 0.98955
TPR: 0.3
TNRxTPR: 0.29686
G-mean: 0.54485
__________________________________________________
Classifier:  LGBMClassifier
[[286   1]
 [  9   1]]

TNR: 0.99652
TPR: 0.1
TNRxTPR: 0.09965
G-mean: 0.31568
__________________________________________________
Classifier:  XGBClassifier
[[285   2]
 [  7   3]]

TNR: 0.99303
TPR: 0.3
TNRxTPR: 0.29791
G-mean: 0.54581
__________________________________________________
Classifier:  BaggingClassifier
[[284   3]
 [  7   3]]

TNR: 0.98955
TPR: 0.3
TNRxTPR: 0.29686
G-mean: 0.54485
__________________________________________________
Classifier:  RandomForestClassifier
[[287   0]
 [  9   1]]

TNR: 1.0
TPR: 0.1
TNRxTPR: 0.1
G-mean: 0.31623
__________________________________________________
Ensemble predictions (majority voting):
[[286   1]
 [  9   1]]

TNR: 0.99652
TPR: 0.1
TNRxTPR: 0.09965
G-mean: 0.31568


## Ratio 1:1  
Let's see classifiers scores when dataset is balanced.  


In [11]:
# SMOTE

sm = SMOTE(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = sm.fit_resample(X_train, y_train)

In [12]:
print('Resampled dataset shape %s' % Counter(y_res))
print('Ratio->  1 :', round(Counter(y_res)[1]/Counter(y_res)[0], 1))

Resampled dataset shape Counter({0: 1146, 1: 1146})
Ratio->  1 : 1.0


In [13]:
train_val(X_res, y_res, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[273  14]
 [  6   4]]

TNR: 0.95122
TPR: 0.4
TNRxTPR: 0.38049
G-mean: 0.61684
__________________________________________________
Classifier:  LGBMClassifier
[[276  11]
 [  7   3]]

TNR: 0.96167
TPR: 0.3
TNRxTPR: 0.2885
G-mean: 0.53712
__________________________________________________
Classifier:  XGBClassifier
[[274  13]
 [  5   5]]

TNR: 0.9547
TPR: 0.5
TNRxTPR: 0.47735
G-mean: 0.69091
__________________________________________________
Classifier:  BaggingClassifier
[[267  20]
 [  5   5]]

TNR: 0.93031
TPR: 0.5
TNRxTPR: 0.46516
G-mean: 0.68202
__________________________________________________
Classifier:  RandomForestClassifier
[[275  12]
 [  5   5]]

TNR: 0.95819
TPR: 0.5
TNRxTPR: 0.47909
G-mean: 0.69217
__________________________________________________
Ensemble predictions (majority voting):
[[272  15]
 [  5   5]]

TNR: 0.94774
TPR: 0.5
TNRxTPR: 0.47387
G-mean: 0.68838


In [14]:
# ROS

ros = RandomOverSampler(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = ros.fit_resample(X_train, y_train)

In [15]:
print('Resampled dataset shape %s' % Counter(y_res))
print('Ratio->  1 :', round(Counter(y_res)[1]/Counter(y_res)[0], 1))

Resampled dataset shape Counter({0: 1146, 1: 1146})
Ratio->  1 : 1.0


In [16]:
train_val(X_res, y_res, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[277  10]
 [  7   3]]

TNR: 0.96516
TPR: 0.3
TNRxTPR: 0.28955
G-mean: 0.5381
__________________________________________________
Classifier:  LGBMClassifier
[[282   5]
 [  8   2]]

TNR: 0.98258
TPR: 0.2
TNRxTPR: 0.19652
G-mean: 0.4433
__________________________________________________
Classifier:  XGBClassifier
[[282   5]
 [  7   3]]

TNR: 0.98258
TPR: 0.3
TNRxTPR: 0.29477
G-mean: 0.54293
__________________________________________________
Classifier:  BaggingClassifier
[[284   3]
 [  8   2]]

TNR: 0.98955
TPR: 0.2
TNRxTPR: 0.19791
G-mean: 0.44487
__________________________________________________
Classifier:  RandomForestClassifier
[[284   3]
 [  8   2]]

TNR: 0.98955
TPR: 0.2
TNRxTPR: 0.19791
G-mean: 0.44487
__________________________________________________
Ensemble predictions (majority voting):
[[282   5]
 [  7   3]]

TNR: 0.98258
TPR: 0.3
TNRxTPR: 0.29477
G-mean: 0.54293


In [17]:
# VAEOversampler

vae_sampler = VAEOversampler(epochs=500,
                              intermediate_dim=512,
                              batch_size=64,
                              random_state=42,
                              verbose=False)
X_res, y_res = vae_sampler.fit_resample(X_train, y_train)

2023-07-11 00:54:56.366557: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-07-11 00:54:56.366613: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-11 00:54:56.366657: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (a-Modern-15-A5M): /proc/driver/nvidia/version does not exist
2023-07-11 00:54:56.367016: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
print('Resampled dataset shape %s' % Counter(y_res))
print('Ratio->  1 :', round(Counter(y_res)[1]/Counter(y_res)[0], 1))

Resampled dataset shape Counter({0.0: 1146, 1.0: 1146})
Ratio->  1 : 1.0


In [19]:
train_val(X_res, y_res, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[285   2]
 [  8   2]]

TNR: 0.99303
TPR: 0.2
TNRxTPR: 0.19861
G-mean: 0.44565
__________________________________________________
Classifier:  LGBMClassifier
[[283   4]
 [  8   2]]

TNR: 0.98606
TPR: 0.2
TNRxTPR: 0.19721
G-mean: 0.44409
__________________________________________________
Classifier:  XGBClassifier
[[283   4]
 [  7   3]]

TNR: 0.98606
TPR: 0.3
TNRxTPR: 0.29582
G-mean: 0.54389
__________________________________________________
Classifier:  BaggingClassifier
[[286   1]
 [  8   2]]

TNR: 0.99652
TPR: 0.2
TNRxTPR: 0.1993
G-mean: 0.44643
__________________________________________________
Classifier:  RandomForestClassifier
[[285   2]
 [  9   1]]

TNR: 0.99303
TPR: 0.1
TNRxTPR: 0.0993
G-mean: 0.31512
__________________________________________________
Ensemble predictions (majority voting):
[[285   2]
 [  9   1]]

TNR: 0.99303
TPR: 0.1
TNRxTPR: 0.0993
G-mean: 0.31512


## Under/Oversampling combination  
Now we can tuning the number of instances for each class to optimize metric.  


In [58]:
# SMOTE

sm = SMOTE(random_state=42, sampling_strategy=.5) # N_rm / N_M
X_res, y_res = sm.fit_resample(X_train, y_train)

In [59]:
# RUS

X_eq, y_eq = RUS(X_res, y_res, frac=.1)

print('Resampled dataset shape %s' % Counter(y_eq))
print('Ratio->  1 :', round(Counter(y_eq)[1]/Counter(y_eq)[0], 1))

Resampled dataset shape Counter({1: 573, 0: 115})
Ratio->  1 : 5.0


In [60]:
train_val(X_eq, y_eq, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[217  70]
 [  1   9]]

TNR: 0.7561
TPR: 0.9
TNRxTPR: 0.68049
G-mean: 0.82492
__________________________________________________
Classifier:  LGBMClassifier
[[221  66]
 [  2   8]]

TNR: 0.77003
TPR: 0.8
TNRxTPR: 0.61603
G-mean: 0.78487
__________________________________________________
Classifier:  XGBClassifier
[[226  61]
 [  2   8]]

TNR: 0.78746
TPR: 0.8
TNRxTPR: 0.62997
G-mean: 0.7937
__________________________________________________
Classifier:  BaggingClassifier
[[226  61]
 [  0  10]]

TNR: 0.78746
TPR: 1.0
TNRxTPR: 0.78746
G-mean: 0.88739
__________________________________________________
Classifier:  RandomForestClassifier
[[221  66]
 [  2   8]]

TNR: 0.77003
TPR: 0.8
TNRxTPR: 0.61603
G-mean: 0.78487
__________________________________________________
Ensemble predictions (majority voting):
[[221  66]
 [  2   8]]

TNR: 0.77003
TPR: 0.8
TNRxTPR: 0.61603
G-mean: 0.78487


In [67]:
# ROS

ros = RandomOverSampler(random_state=42, sampling_strategy=.5) # N_rm / N_M
X_res, y_res = ros.fit_resample(X_train, y_train)

In [68]:
# RUS

X_eq, y_eq = RUS(X_res, y_res, frac=.1)

print('Resampled dataset shape %s' % Counter(y_eq))
print('Ratio->  1 :', round(Counter(y_eq)[1]/Counter(y_eq)[0], 1))

Resampled dataset shape Counter({1: 573, 0: 115})
Ratio->  1 : 5.0


In [69]:
train_val(X_eq, y_eq, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[223  64]
 [  0  10]]

TNR: 0.777
TPR: 1.0
TNRxTPR: 0.777
G-mean: 0.88148
__________________________________________________
Classifier:  LGBMClassifier
[[240  47]
 [  3   7]]

TNR: 0.83624
TPR: 0.7
TNRxTPR: 0.58537
G-mean: 0.76509
__________________________________________________
Classifier:  XGBClassifier
[[232  55]
 [  3   7]]

TNR: 0.80836
TPR: 0.7
TNRxTPR: 0.56585
G-mean: 0.75223
__________________________________________________
Classifier:  BaggingClassifier
[[234  53]
 [  0  10]]

TNR: 0.81533
TPR: 1.0
TNRxTPR: 0.81533
G-mean: 0.90296
__________________________________________________
Classifier:  RandomForestClassifier
[[239  48]
 [  3   7]]

TNR: 0.83275
TPR: 0.7
TNRxTPR: 0.58293
G-mean: 0.7635
__________________________________________________
Ensemble predictions (majority voting):
[[229  58]
 [  2   8]]

TNR: 0.79791
TPR: 0.8
TNRxTPR: 0.63833
G-mean: 0.79895


In [147]:
# VAEOversampler

X_res, y_res = vae_sampler.resample(X_train, y_train, sampling_strategy=3)

In [148]:
# RUS

X_eq, y_eq = RUS(X_res, y_res, frac=.04)

print('Resampled dataset shape %s' % Counter(y_eq))
print('Ratio->  1 :', round(Counter(y_eq)[1]/Counter(y_eq)[0], 1))

Resampled dataset shape Counter({1.0: 3356, 0.0: 46})
Ratio->  1 : 73.0


In [149]:
train_val(X_eq, y_eq, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[242  45]
 [  0  10]]

TNR: 0.84321
TPR: 1.0
TNRxTPR: 0.84321
G-mean: 0.91826
__________________________________________________
Classifier:  LGBMClassifier
[[218  69]
 [  3   7]]

TNR: 0.75958
TPR: 0.7
TNRxTPR: 0.53171
G-mean: 0.72918
__________________________________________________
Classifier:  XGBClassifier
[[195  92]
 [  1   9]]

TNR: 0.67944
TPR: 0.9
TNRxTPR: 0.6115
G-mean: 0.78198
__________________________________________________
Classifier:  BaggingClassifier
[[189  98]
 [  3   7]]

TNR: 0.65854
TPR: 0.7
TNRxTPR: 0.46098
G-mean: 0.67895
__________________________________________________
Classifier:  RandomForestClassifier
[[206  81]
 [  1   9]]

TNR: 0.71777
TPR: 0.9
TNRxTPR: 0.64599
G-mean: 0.80374
__________________________________________________
Ensemble predictions (majority voting):
[[200  87]
 [  1   9]]

TNR: 0.69686
TPR: 0.9
TNRxTPR: 0.62718
G-mean: 0.79195


## References  

  - Classification with Imbalanced Datasets:  
    https://sci2s.ugr.es/imbalanced  
  - Computer Vision:  Models, Learning, and Inference (Simon J.D. Prince):  
    http://www.computervisionmodels.com/  
  - Oversampling with VAEs:  
    https://towardsdatascience.com/oversampling-with-vaes-e410887fe51  
